## 텍스트 마이닝
# 사우스파크 챗봇
- kaggle 에서 south park 로 검색

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
spark_df = pd.read_csv('south_park_chatbot/All-seasons.csv')
spark_df.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


In [5]:
spark_df.tail()

,Season,Episode,Character,Line
70891,9,14,Stan,I think you're pushing it.\n
70892,9,14,Randy,How about twenty?\n
70893,9,14,Stan,That's not disciprine.\n
70894,9,14,Randy,Right right. Does vodka count?\n
70895,9,14,Stan,Dad!\n


In [4]:
spark_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70896 entries, 0 to 70895
Data columns (total 4 columns):
Season       70896 non-null object
Episode      70896 non-null object
Character    70896 non-null object
Line         70896 non-null object
dtypes: object(4)
memory usage: 2.2+ MB


In [6]:
spark_df.Season.unique()

array(['10', 'Season', '11', '12', '13', '14', '15', '16', '17', '18',
       '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)

In [7]:
spark_df[spark_df.Season == 'Season']

,Season,Episode,Character,Line
3471,Season,Episode,Character,Line
6950,Season,Episode,Character,Line
10258,Season,Episode,Character,Line
13516,Season,Episode,Character,Line
16863,Season,Episode,Character,Line
19965,Season,Episode,Character,Line
23086,Season,Episode,Character,Line
25392,Season,Episode,Character,Line
27915,Season,Episode,Character,Line
32086,Season,Episode,Character,Line


In [12]:
spark_df.Character.value_counts().head(10)

Cartman         9774
Stan            7680
Kyle            7099
Butters         2602
Randy           2467
Mr. Garrison    1002
Chef             917
Kenny            881
Sharon           862
Mr. Mackey       633
Name: Character, dtype: int64

In [22]:
spark_df.Line.apply(len).max() # 3(min) ~ 1585(max)

1585

In [23]:
print(spark_df.iloc[spark_df.Line.apply(len).idxmax(),3])
print(spark_df.iloc[spark_df.Line.apply(len).idxmin(),3])

Okay, first of all, I am NOT Aquaman, I am a recovering... gay fish. Yes, I have met Aquaman. I have hung out with Aquaman. But the only thing I have in common with Aquaman anymore is my love for the sea.  Now!  There have been malicious rumors, started at this elementary school, that my beautiful fiancée is a hobbit. That is not funny, and it is not true.  Alright?! Yes, Kim is heavier than most of her pictures show her to be. Yes, she gets her hair lasered off her body. Yes, she has a friend named Gandalf who happens to be a wizard.  I'm sorry, excuse me a minute.  Bitch, how are you not a hobbit again? Yeah. Yeah yeah yeah right. Rightrightrightrightright, yeah. Okay. Yep. Yep. Let me get- okay. Yep, I got it. Okay, love you too.  Okay, if my fiancée Kim… is… a hobbit, then how come… it uh how c- then, okay, if she's a hobbit, then how come she don't live in a hole in the ground? BOOOOOOM! All y'all just got lit up, cuz! She don't live in no hole in the ground, she lives in a big-as

In [35]:
import re

def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"\n", "",  text)
    text = re.sub(r"[-()]", "", text)
    text = re.sub(r"\.", " .", text)
    text = re.sub(r"\!", " !", text)
    text = re.sub(r"\?", " ?", text)
    text = re.sub(r"\,", " ,", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"ohh", "oh", text)
    text = re.sub(r"ohhh", "oh", text)
    text = re.sub(r"ohhhh", "oh", text)
    text = re.sub(r"ohhhhh", "oh", text)
    text = re.sub(r"ohhhhhh", "oh", text)
    text = re.sub(r"ahh", "ah", text)
    
    return text

text = []

for line in spark_df.Line:
    text.append(clean_text(line))

In [36]:
len(text)

70896

In [38]:
lengths = [len(s.split()) for s in text]
len(lengths)

70896

In [64]:
max_line_length = 30

short_text = [s for s in text if len(s.split())<=max_line_length]
len(short_text)

63991

In [65]:
vocab = {}
for s in short_text:
    for w in s.split():
        if w in vocab: vocab[w] += 1
        else: vocab[w] = 1
        
len(vocab)

24075

In [66]:
vocab2 = vocab.copy()
for k,v in vocab2.items():
    if v<3: del vocab[k]
        
len(vocab)

8333

In [67]:
n=0
for k in vocab:
    print(k+' : '+str(vocab[k]))
    if n>=10: break
    n += 1

you : 19705
guys : 1503
, : 42194
! : 34689
chef : 354
is : 14211
going : 1319
away : 412
. : 57630
? : 19930
for : 3364


In [68]:
vocab2int = {}
for i, (k,v) in enumerate(vocab.items()):
    vocab2int[k] = i
    
len(vocab2int)

8333

In [76]:
codes = ['<PAD>','<EOS>','<UNK>','<GO>']
vocab_len = len(vocab2int)
for i in range(4):
    vocab2int[codes[i]] = vocab_len + i
    
len(vocab2int)

8337

In [77]:
int2vocab = {v:k for k,v in vocab2int.items()}
len(int2vocab)

8337

### 벡터화

In [78]:
short_text[:5]

['you guys , you guys ! chef is going away . ',
 'going away ? for how long ?',
 'forever .',
 'i am sorry boys .',
 'chef said he is been bored , so he joining a group called the super adventure club . ']

In [84]:
source_text = short_text[:-1]
target_text = short_text[1:]

In [85]:
vocab_len = len(vocab2int)

source_vector = []
for s in source_text:
    l = []
    for w in s.split():
        if w in vocab2int: l.append(vocab2int[w])
        else: l.append(vocab_len-2) # '<UNK>'
    source_vector.append(l)
    
target_vector = []
for s in target_text:
    l = []
    for w in s.split():
        if w in vocab2int: l.append(vocab2int[w])
        else: l.append(vocab_len-2) # '<UNK>'
    target_vector.append(l+[vocab_len-3]) # <'EOS'>

In [86]:
len(source_vector), len(target_vector)

(63990, 63990)

In [92]:
source_vector[1], target_vector[0]

([6, 7, 9, 10, 11, 12, 9], [6, 7, 9, 10, 11, 12, 9, 8334])

### 데이터 정리
- source_vector : 입력 문장의 시퀀스, 63990개 (한 시퀀스의 길이는 1 ~ 30)
- target_vector : 출력 문장의 시퀀스, 63990개, 끝에 ```<EOS>``` 추가, (한 시퀀스의 길이는 2 ~ 31)
- vocab2int : 단어-인덱스 사전, 8337개 단어, 끝에 ```<PAD>,<EOS>,<UNK>,<GO>```
- int2vocab : 인덱스-단어 사전, 8337개 단어

### 케라스 테스트

In [95]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Embedding(8337, 8, input_length=None)) # 단어갯수, 변환차원, 시퀀스길이
# keras.preprocessing.sequence.pad_sequences() 이용해 시퀀스 길이를 제한할 수 있다

In [96]:
help(Embedding)

Help on class Embedding in module keras.layers.embeddings:

class Embedding(keras.engine.base_layer.Layer)
 |  Turns positive integers (indexes) into dense vectors of fixed size.
 |  eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]
 |  
 |  This layer can only be used as the first layer in a model.
 |  
 |  # Example
 |  
 |  ```python
 |    model = Sequential()
 |    model.add(Embedding(1000, 64, input_length=10))
 |    # the model will take as input an integer matrix of size (batch, input_length).
 |    # the largest integer (i.e. word index) in the input should be
 |    # no larger than 999 (vocabulary size).
 |    # now model.output_shape == (None, 10, 64), where None is the batch dimension.
 |  
 |    input_array = np.random.randint(1000, size=(32, 10))
 |  
 |    model.compile('rmsprop', 'mse')
 |    output_array = model.predict(input_array)
 |    assert output_array.shape == (32, 10, 64)
 |  ```
 |  
 |  # Arguments
 |      input_dim: int > 0. Size of the vocabulary,
 |          i.